In [4]:
import pandas as pd
import torch
import numpy as np
from keras.utils import to_categorical

2023-06-06 15:35:16.083477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 15:35:16.705242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
data_org = pd.read_csv('~/Downloads/final_data.csv')
data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13806,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13807,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13808,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


In [7]:
sum_vec = data_org.iloc[:, :9].sum(axis=1)
data2 = data_org.iloc[:, :9].divide(sum_vec, axis='index')
data = pd.concat([data2, data_org['seq']], axis=1)
data

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...


In [8]:
data['seq'][0]
codetable = np.zeros(256,np.int64)
for ix,nt in enumerate(['A','C','G','T']):
  codetable[ord(nt)] = ix
categorical_vector = codetable[np.array(list(seq)).view(np.int32)]
one_hot = torch.nn.functional.one_hot(torch.from_numpy(categorical_vector), num_classes=4)

NameError: name 'seq' is not defined

In [15]:
seq = 'ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGAT'
genes = ['A', 'C', 'G', 'T']
liste = pd.DataFrame(list(seq))

mapping = {}
for x in range(len(genes)):
  mapping[genes[x]] = x
  
for x in range(len(seq)):
  seq[x] = mapping[seq[x]]
  
one_hot_encode = to_categorical(seq)

,0
0,A
1,C
2,C
3,T
4,T
5,G
6,T
7,A
8,T
9,T
